In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from shutil import copyfile
from sklearn.model_selection import train_test_split
from IPython.display import SVG,display
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, Adam
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.utils import to_categorical
from keras.applications import VGG16, DenseNet201, ResNet50, InceptionV3, InceptionResNetV2
from keras.applications.vgg16 import decode_predictions
from keras.preprocessing.image import load_img
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

/home/diegues/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def loadModel(modelname, optimizer):
    if('dense' in modelname):
        print('densenet')
        model = DenseNet201()
    elif('res' in modelname):
        print('resnet')
        model = ResNet50()
    elif('inceptionv3' in modelname):
        print('inceptionv3')
        model = InceptionV3()
    else:
        print('inceptionresnet')
        model = InceptionResNetV2()
        
    x = Dense(5, activation='softmax', name='predictions')(model.layers[-2].output)

    M = Model(input=model.input, output=x)
    M.load_weights(modelname)
    M.compile(loss='categorical_crossentropy',
                optimizer=optimizer, 
                metrics=['acc'])
    return M
    

In [3]:
folder = '/media/newdrive/20180501/ProcessedImages/' # change this path to the one you want
test_path = folder + 'test/' # change this path to the one you want
test_df = pd.read_csv(folder + 'unknown2-targets.csv').set_index('filename') # change this path to the one you want
test_datagen = ImageDataGenerator(rescale=1./255)

In [4]:
preds = {}
weights_folder = '/home/diegues/git/AutomaticHabitatMapping/src/weights/' # change this path to the one you want
names = [f for f in os.listdir(weights_folder)]
for name in names:
    if('vgg' in name): 
        continue
    if('nesterov' in name):
        optimizer = SGD(lr = 0.001, nesterov=True)
    else:
        optimizer = SGD(lr=0.001)
    cnn = loadModel(weights_folder + name, optimizer)

    if('inception' in name):
        target_size = (299, 299)
    else:
        target_size = (224, 224)

    test_generator = test_datagen.flow_from_directory(
            test_path,
            target_size=target_size,
            batch_size=1,
            class_mode=None,
            shuffle=False)
    preds[name] = cnn.predict_generator(test_generator, 
                          steps=751, verbose=1)

resnet


/home/diegues/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("pr...)`


Found 751 images belonging to 1 classes.
751/751 [==============================] - 19s 25ms/step
densenet
Found 751 images belonging to 1 classes.
751/751 [==============================] - 43s 57ms/step
inceptionv3
Found 751 images belonging to 1 classes.
751/751 [==============================] - 26s 34ms/step


In [5]:
preds

{'densenet_0_5.h5': array([[4.6120551e-01, 4.9110910e-01, 3.4763999e-02, 7.0095751e-03,
         5.9118313e-03],
        [4.8661762e-01, 5.1132423e-01, 1.1806266e-03, 1.3327734e-04,
         7.4424903e-04],
        [7.6744419e-01, 2.1350206e-01, 9.7181909e-03, 4.6579815e-03,
         4.6775546e-03],
        ...,
        [9.0716195e-01, 3.8934667e-02, 1.4641772e-03, 5.1192239e-02,
         1.2470395e-03],
        [8.1474966e-01, 9.4687656e-02, 1.9881288e-03, 8.5096866e-02,
         3.4776775e-03],
        [3.1355521e-01, 3.8456386e-01, 2.2119708e-02, 2.5595227e-01,
         2.3808941e-02]], dtype=float32),
 'inceptionv3_0_5.h5': array([[0.48203582, 0.286037  , 0.06999105, 0.1221614 , 0.03977476],
        [0.35677874, 0.53785664, 0.02206844, 0.05113003, 0.03216613],
        [0.3374777 , 0.37493646, 0.04040605, 0.20148447, 0.04569531],
        ...,
        [0.3586584 , 0.24180435, 0.09092722, 0.18147032, 0.12713973],
        [0.17439285, 0.14983356, 0.05160796, 0.55139494, 0.07277063],
  

In [6]:
for key in preds.keys():
    cnn = pd.DataFrame(preds[key]).set_index(test_df.index)
    cnn.columns = ['A3.1-A4.1', 'A3.7-A4.7', 'A5.1', 'A5.2', 'A5.4']
    cnn['level3'] = cnn.idxmax(axis=1)
    preds[key] = cnn

In [7]:
res = pd.DataFrame()
for key in preds.keys():
    res[key] = preds[key].level3
res['level3'] = test_df.level3.replace(['A3.1', 'A4.1'], 'A3.1-A4.1').replace(['A3.7', 'A4.7'], 'A3.7-A4.7')
res.head()

,resnet_0_5.h5,densenet_0_5.h5,inceptionv3_0_5.h5,level3
filename,,,,
105317_cam-np3_frame1347.jpg,A3.7-A4.7,A3.7-A4.7,A3.1-A4.1,A3.7-A4.7
105317_cam-np3_frame1352.jpg,A3.7-A4.7,A3.7-A4.7,A3.7-A4.7,A3.7-A4.7
105317_cam-np3_frame1390.jpg,A3.7-A4.7,A3.1-A4.1,A3.7-A4.7,A3.1-A4.1
105317_cam-np3_frame1468.jpg,A3.7-A4.7,A3.1-A4.1,A3.1-A4.1,A3.1-A4.1
105317_cam-np3_frame1473.jpg,A3.7-A4.7,A3.1-A4.1,A3.1-A4.1,A3.1-A4.1


In [8]:
for col in res.columns[:-1]:
    print(col)
    print(accuracy_score(res.level3, res[col]))#, precision_score(res.level3, res[col]), recall_score(res.level3, res[col]), f1_score(res.level3, res[col]))
    print(classification_report(res.level3, res[col]))
    

resnet_0_5.h5
0.6498002663115846
             precision    recall  f1-score   support

  A3.1-A4.1       0.71      0.72      0.72       408
  A3.7-A4.7       0.42      0.67      0.52       128
       A5.1       0.85      0.66      0.74       125
       A5.2       0.65      0.35      0.45        49
       A5.4       1.00      0.20      0.33        41

avg / total       0.69      0.65      0.65       751

densenet_0_5.h5
0.7163781624500666
             precision    recall  f1-score   support

  A3.1-A4.1       0.72      0.90      0.80       408
  A3.7-A4.7       0.64      0.48      0.55       128
       A5.1       0.82      0.65      0.72       125
       A5.2       0.53      0.35      0.42        49
       A5.4       1.00      0.24      0.39        41

avg / total       0.72      0.72      0.70       751

inceptionv3_0_5.h5
0.6591211717709721
             precision    recall  f1-score   support

  A3.1-A4.1       0.68      0.84      0.75       408
  A3.7-A4.7       0.46      0.38      0

# VGG16 with Dropout

In [26]:
vgg = VGG16(weights='imagenet')
fc1 = vgg.layers[-3]
fc2 = vgg.layers[-2]
predictions = Dense(5, activation='softmax', name='predictions')


dropout1 = Dropout(0.5)
dropout2 = Dropout(0.5)

x = dropout1(fc1.output)
x = fc2(x)
x = dropout2(x)
predictors = predictions(x)

vgg16 = Model(input=vgg.input, output=predictors)
vgg16.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

/home/diegues/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("pr...)`
  from ipykernel import kernelapp as app


In [40]:
name = '/home/diegues/git/AutomaticHabitatMapping/src/best_weights/vgg16_nesterov_cp.h5' # change this path to the one you want
vgg16.load_weights(name)
if('nesterov' in name):
    optimizer = SGD(lr = 0.001, nesterov=True)
else:
    optimizer = SGD(lr=0.001)

vgg16.compile(loss='categorical_crossentropy',
            optimizer=optimizer, 
            metrics=['acc'])

test_generator = test_datagen.flow_from_directory(
        test_path,
        target_size=(224,224),
        batch_size=1,
        class_mode=None,
        shuffle=False)
predictions = vgg16.predict_generator(test_generator, 
                      steps=751, verbose=1)

Found 751 images belonging to 1 classes.
751/751 [==============================] - 43s 57ms/step


In [41]:
_preds = pd.DataFrame(predictions).set_index(test_df.index)
_preds.columns = ['A3.1-A4.1', 'A3.7-A4.7', 'A5.1', 'A5.2', 'A5.4']
_preds['level3'] = _preds.idxmax(axis=1)

In [42]:
print(name.split('/')[-1])
print(accuracy_score(res.level3, _preds.level3))
print(classification_report(res.level3, _preds.level3))

vgg16_nesterov_cp.h5
0.33422103861517977
             precision    recall  f1-score   support

  A3.1-A4.1       0.53      0.26      0.35       408
  A3.7-A4.7       0.27      0.49      0.35       128
       A5.1       0.26      0.50      0.34       125
       A5.2       0.09      0.06      0.07        49
       A5.4       0.38      0.34      0.36        41

avg / total       0.40      0.33      0.33       751

